# Importations et chargement des données

In [2]:
from mlflow.models.signature import infer_signature
import pandas as pd
from sklearn.model_selection import train_test_split
import mlflow.sklearn
from joblib import load
import os

In [3]:
data_dir = "./home-credit-default-risk"
app = pd.read_csv(data_dir+'/application_train_preprocessed.csv')

In [4]:
target = app['TARGET']
app.drop(columns=['TARGET'], inplace=True)

In [5]:
app = app.set_index('SK_ID_CURR')

In [6]:
X_train, X_test, y_train, y_test = train_test_split(app.to_numpy(), target.to_numpy(), test_size=0.2)

# Sauvegarde du modèle avec Mlflow   
Attention, le répertoire mlflow_model doit être supprimé avant

In [11]:
signature = infer_signature(X_train, y_train)

On charge le modèle à partir de notre fichier joblib

In [13]:
model = load('./joblib/adaboost.joblib')

On modifie la fonction predict pour qu'elle nous retourne la probabilité. En effet, le modèle Mlflow ne gère que cette fonction.

In [21]:
model.predict = model.predict_proba

In [22]:
mlflow.sklearn.save_model(model, 'mlflow_model', signature=signature)

# Essai sur un exemple

On extrait les features en entrée du premier individus

In [18]:
str([round(x, 2) for x in X_train[0]])

'[1.05, -0.15, -0.75, -1.75, -1.31, 0.0, -0.55, -1.27, -1.02, -0.5, 0.21, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.74, 17.64, 4.31, 0.11, 0.0, 0.0, 0.0, 10.0, 0.56, 0.55, -0.34, -0.16, 0.36, 0.51, 0.85, -0.72, -1.06, -0.55]'

On démarre notre serveur avec la commande suivante sur bash:

mlflow models serve -m mlflow_model/

Reste à entrer la requête suivante dans un autre terminal:

curl http://127.0.0.1:5000/invocations -H 'Content-Type: application/json' -d '{"data": [[1.05, -0.15, -0.75, -1.75, -1.31, 0.0, -0.55, -1.27, -1.02, -0.5, 0.21, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.74, 17.64, 4.31, 0.11, 0.0, 0.0, 0.0, 10.0, 0.56, 0.55, -0.34, -0.16, 0.36, 0.51, 0.85, -0.72, -1.06, -0.55]]}'

On peut aussi faire l'essai à partir du notebook

In [32]:
res = os.popen('curl http://127.0.0.1:5000/invocations -H \'Content-Type: application/json\' -d \'{"data": [[1.05, -0.15, -0.75, -1.75, -1.31, 0.0, -0.55, -1.27, -1.02, -0.5, 0.21, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.74, 17.64, 4.31, 0.11, 0.0, 0.0, 0.0, 10.0, 0.56, 0.55, -0.34, -0.16, 0.36, 0.51, 0.85, -0.72, -1.06, -0.55]]}\'')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   288  100    42  100   246   5167  30269 --:--:-- --:--:-- --:--:-- 36000


In [33]:
res = res.read()
res_ = res.split(',')[1]
res_ = float(res_[:-2])
res_